In [2]:
from glob import glob
import pandas as pd
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedShuffleSplit
import itertools
from collections import defaultdict

all_model_scores = defaultdict(list)
#print(1)

# For every data set in the data repository...
for dataset in glob('data/ecoli.csv.gz'):
    # Read the data set into memory
    input_data = pd.read_csv(dataset, compression='gzip')
    
    for dataset_repeat in range(1, 31):
        # Divide the data set into a training and testing sets, each time with a different RNG seed
        training_indices, testing_indices = next(iter(StratifiedShuffleSplit(input_data['class'].values,
                                                                             n_iter=1,
                                                                             train_size=0.75,
                                                                             test_size=0.25,
                                                                             random_state=dataset_repeat)))
        
        training_features = input_data.loc[training_indices].drop('class', axis=1).values
        training_classes = input_data.loc[training_indices, 'class'].values

        testing_features = input_data.loc[testing_indices].drop('class', axis=1).values
        testing_classes = input_data.loc[testing_indices, 'class'].values
        
        # These are parameters listed in the project document
        C_list = [0.01, 0.1, 0.5, 1.0, 10.0, 50.0, 100.0]
        gamma_list = [0.01, 0.1, 0.5, 1.0, 10.0, 50.0, 100.0]
        shrinking_list = [True, False]
        kernel_list = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
        #degree_list = [[2,3,4] for x in kernel_list if x=='poly']
        # For every DecisionTreeClassifier parameter combination...
        for (C, gamma, shrinking, kernel) in itertools.product(C_list, gamma_list, shrinking_list, kernel_list):
            # Create and fit the model on the training data
            clf = SVC(C = C, gamma = gamma, shrinking = shrinking, kernel = kernel)
            clf.fit(training_features, training_classes)
            
            # Store the model's score with the key (name of data set, model, model parameters)
            all_model_scores[(dataset,
                              'SVC',
                              (C, gamma, shrinking, kernel))].append(clf.score(testing_features,
                                                                                      testing_classes))

ValueError: X.shape[0] should be equal to X.shape[1]